In [47]:
import openai
import json
import os

In [48]:
client = openai.OpenAI(api_key = "XXXXXXXXX") #Put your own open ai api
directory = 'data'
model = 'gpt-4'


In [49]:
def generatePrompt(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        data = json.load(file)
    data_str = json.dumps(data, ensure_ascii=False, indent=4)

    splitPoint = len(data_str) // 2
    gpt_user_prompt = ("This is a json containing information of a faculty website,"
                    "find out all names of faculty, their position,"
                    "if there is a link connect to it, also include it. If no link, write Null"
                    "Also, Only include unique names. Return in json format as well"
                    "Don't return anything other than the json file")
    partOne = gpt_user_prompt + data_str[:splitPoint]
    partTwo = gpt_user_prompt + data_str[splitPoint:]
    return [partOne, partTwo]

def getResponse(gpt_user_prompt):
    message=[{"role": "user", "content": gpt_user_prompt}]
    temperature=0.2
    max_tokens=3600
    frequency_penalty=0.0
    response = client.chat.completions.create(
        model=model,
        messages = message,
        stream=False,
        temperature=temperature,
        max_tokens=max_tokens,
        frequency_penalty=frequency_penalty
    )
    return response.choices[0].message.content

def saveData(filename, id, response):
    newName = filename.replace('data/', 'result/')
    newName = newName.replace('.json', str(id)+'.json')
    newName = newName.replace('extracted', 'analyzed')
    with open(newName, 'w', encoding='utf-8') as file:
        file.write(response)
        print(newName + ' successed')


In [51]:
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        try:
            prompts = generatePrompt(os.path.join(directory, filename))
            for i in range(len(prompts)):
                response = getResponse(prompts[i])
                saveData(os.path.join(directory, filename), i, response)
        except:
            print(os.path.join(directory, filename) + ' failed')

result/analyzed_https:__geography.wisc.edu_people_faculty_0.json successed
result/analyzed_https:__geography.wisc.edu_people_faculty_1.json successed
result/analyzed_https:__geog.ufl.edu_faculty_0.json successed
data/extracted_https:__geog.ufl.edu_faculty_.json failed
